In [23]:
# 1. external function(sigmoid, numerical_derivative)

import numpy as np

# sigmoid 함수

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 수치미분 함수

def numerical_derivative(f, x): #x는 모든 변수를 포함하고 있는 numpy객체(배열, 행렬...)
    delta_x = 1e-4 #0.0001
    grad = np.zeros_like(x)

    it = np.nditer(x, flags = ['multi_index'], op_flags=['readwrite']) #모든 입력변수에 대해 편미분하기 위해 iterator 사용
    
    while not it.finished: 
        idx = it.multi_index
        
        tmp_val = x[idx] 
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) #f(x+delta_x)
        
        x[idx] = tmp_val - delta_x
        fx2 = f(x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val
        it.iternext()
            
    return grad


In [30]:
# 2. LogicGate Class (__Init__, feed_forward, loss_val)

class LogicGate:
    def __init__(self, gate__name, xdata, tdata):
        self.name = gate__name

        #입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4, 2) # 4개의 입력데이터 x1, x2 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4, 1) # 4개의 입력데이터 x1, x2에 대한 각각의 계산 값 행렬

        #2층 hidden layer unit : 6개 가정, 가중치 w2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2, 6) #weight, 2x6 matrix
        self.__b2 = np.random.rand(6)

        #3층 output layer unit: 1개, 가중치 w3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(6, 1)
        self.__b3 = np.random.rand(1)

        # 학습을 learning rate 초기화
        self.__learning_rate = 1e-2

        print(self.name + " object is created")
    
    def feed_forward(self):  #feed forward를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7 # log 무한대 발산 방지
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  #은닉층의 선형회귀 값
        a2 = sigmoid(z2)  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3) # 출력층의 출력
        
        #cross-entropy
        return -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1-y)+delta))
   
    def loss_val(self): #외부 출력을 위한 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7 # log 무한대 발산 방지
        
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  #은닉층의 선형회귀 값
        a2 = sigmoid(z2)  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3) # 출력층의 출력
        
        #cross-entropy
        return -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1-y)+delta))

    #수치미분을 이용하여 손실함수가 최소가 될 때까지 학습하는 함수
    
    def train(self):
        
        f = lambda x : self.feed_forward()
        print("initial loss value = ", self.loss_val())
        
        for step in range(10001):
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
            
            if (step % 400 == 0):
                print("step = ", step, ", loss value = ", self.loss_val())
                
    #query, 즉 미래값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2 #은닉층의 선형회귀 값
        a2 = sigmoid(z2) # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3 # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3) #출력층의 출력
        
        if y > 0.5:
            result = 1 #True
        else:
            result = 0 #False
        
        return y, result
        

In [32]:
#AND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([ 0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)
and_obj.train()


AND object is created
initial loss value =  10.330512470362411
step =  0 , loss value =  9.98392146871293
step =  400 , loss value =  2.211916998307379
step =  800 , loss value =  2.119409879057746
step =  1200 , loss value =  1.9742533253296117
step =  1600 , loss value =  1.7249297998223008
step =  2000 , loss value =  1.3873023888870264
step =  2400 , loss value =  1.0582092961966647
step =  2800 , loss value =  0.7843550327891918
step =  3200 , loss value =  0.5788182228396406
step =  3600 , loss value =  0.43426317542137055
step =  4000 , loss value =  0.33447097368827394
step =  4400 , loss value =  0.26486052671946864
step =  4800 , loss value =  0.2151793705601941
step =  5200 , loss value =  0.17877275619121846
step =  5600 , loss value =  0.15139347271552983
step =  6000 , loss value =  0.1303062640173091
step =  6400 , loss value =  0.11371490778875858
step =  6800 , loss value =  0.1004122721087402
step =  7200 , loss value =  0.08956776531203309
step =  7600 , loss value =

In [27]:
#AND gate prediction

test_data = np.array([[0,0], [0,1], [1,0], [1,1]])

for data in test_data:
    print(and_obj.predict(data))

(array([7.09927471e-18]), 0)
(array([0.00178238]), 0)
(array([0.00178238]), 0)
(array([0.9868196]), 1)


In [33]:
#OR Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([ 0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)
or_obj.train()


OR object is created
initial loss value =  2.7332207568921176
step =  0 , loss value =  2.716608437393674
step =  400 , loss value =  2.017150637232486
step =  800 , loss value =  1.799994789606453
step =  1200 , loss value =  1.4842372534957249
step =  1600 , loss value =  1.1161449176068485
step =  2000 , loss value =  0.7923412950172185
step =  2400 , loss value =  0.5585843011250852
step =  2800 , loss value =  0.4041381765343627
step =  3200 , loss value =  0.3034782563674649
step =  3600 , loss value =  0.23636832944332306
step =  4000 , loss value =  0.19004072617648388
step =  4400 , loss value =  0.15689196608948291
step =  4800 , loss value =  0.13238109819219362
step =  5200 , loss value =  0.11372780247636774
step =  5600 , loss value =  0.09917504086424317
step =  6000 , loss value =  0.08757612432424616
step =  6400 , loss value =  0.0781596373119834
step =  6800 , loss value =  0.07039204154397251
step =  7200 , loss value =  0.0638949966110363
step =  7600 , loss value 

In [34]:
#OR gate prediction

test_data = np.array([[0,0], [0,1], [1,0], [1,1]])

for data in test_data:
    print(or_obj.predict(data))

(array([0.02158807]), 0)
(array([0.99251764]), 1)
(array([0.99211431]), 1)
(array([0.99948532]), 1)


In [ ]:
# NAND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

In [ ]:
# NAND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(nand_obj.predict(data))

In [37]:
# XOR Gate 객체 생성

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XOR object is created
initial loss value =  7.098920374782523
step =  0 , loss value =  6.938106295829626
step =  400 , loss value =  2.765563533091853
step =  800 , loss value =  2.7602625121080884
step =  1200 , loss value =  2.754279553884655
step =  1600 , loss value =  2.7470638721236527
step =  2000 , loss value =  2.7379555708676415
step =  2400 , loss value =  2.726116321886064
step =  2800 , loss value =  2.7104588379313665
step =  3200 , loss value =  2.689583022195402
step =  3600 , loss value =  2.661743949814735
step =  4000 , loss value =  2.624907973479538
step =  4400 , loss value =  2.576993911754545
step =  4800 , loss value =  2.5163896385672118
step =  5200 , loss value =  2.442642046578754
step =  5600 , loss value =  2.3568174147046372
step =  6000 , loss value =  2.2609116767851942
step =  6400 , loss value =  2.156383335951963
step =  6800 , loss value =  2.04267130557649
step =  7200 , loss value =  1.9165110642651992
step =  7600 , loss value =  1.772781575412

In [36]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.07061648]), 0)
(array([0.85462985]), 1)
(array([0.90451167]), 1)
(array([0.16932362]), 0)
